### libraries

In [1]:
import numpy as np
import cv2
import glob, os
import pytesseract
from PIL import Image, ImageEnhance, ImageFilter,ImageOps
import math
from scipy import ndimage
import imutils
import math

import matplotlib.pyplot as plt
import re
import pandas as pd

import tempfile

# from brisque import BRISQUE

In [2]:
pd.set_option('display.max_columns', None)  # or 1000
pd.set_option('display.max_rows', None)  # or 1000
pd.set_option('display.max_colwidth', -1)
pd.option_context('display.max_rows', None, 'display.max_columns', None)

### Image Rotation 

In [18]:
IMAGE_SIZE = 2000
BINARY_THREHOLD = 175

In [69]:
def rotate(image, center=None, scale=1):

    center=None
    scale=1
    # image = cv2.imread('../data/all_images/processed/image_1_processed.jpg')
    image = cv2.imread(image)
    img = image

    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    gray = cv2.bitwise_not(gray)
    thresh = cv2.threshold(gray, 0, 255,cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]

    '''grab the (x, y) coordinates of all pixel values that are greater than zero, then use these coordinates to
    compute a rotated bounding box that contains all coordinates '''
    coords = np.column_stack(np.where(thresh > 0))
    angle = cv2.minAreaRect(coords)[-1]

    '''the `cv2.minAreaRect` function returns values in the range [-90, 0); as the rectangle rotates clockwise the
    returned angle trends to 0 -- in this special case we need to add 90 degrees to the angle '''
    if angle < -45:
        angle = -(90 + angle)

    # otherwise, just take the inverse of the angle to make it positive
    else:
        angle = -angle
    # rotate the image to deskew it
    (h, w) = img.shape[:2]
    center = (w // 2, h // 2)
    M = cv2.getRotationMatrix2D(center, angle, 1.0)
    rotated = cv2.warpAffine(image, M, (w, h),flags=cv2.INTER_CUBIC, borderMode=cv2.BORDER_REPLICATE)

    gray1 = cv2.cvtColor(rotated, cv2.COLOR_BGR2GRAY)
    (h, w) = gray1.shape[:2]
    #     gray = cv2.bitwise_not(gray)
    scale_percent = 150  # percent of original size
    width = int(gray1.shape[1] * scale_percent / 100)
    height = int(gray1.shape[0] * scale_percent / 100)
    dim = (width, height)
    resized = cv2.resize(gray1, dim, interpolation=cv2.INTER_AREA)
    th2 = cv2.adaptiveThreshold(resized,255,cv2.ADAPTIVE_THRESH_MEAN_C,\
            cv2.THRESH_BINARY,9,15)

    try:
        angle = 360 - int(
            re.search('(?<=Rotate: )\d+',
                      pytesseract.image_to_osd(th2)).group(0))
    except:
        scale_percent = 150  # percent of original size
        width = int(img.shape[1] * scale_percent / 100)
        height = int(img.shape[0] * scale_percent / 100)
        dim = (width, height)
        resized = cv2.resize(img, dim, interpolation=cv2.INTER_AREA)
        angle = 360 - int(
            re.search('(?<=Rotate: )\d+',
                      pytesseract.image_to_osd(resized)).group(0))

    if angle == 90:
        x = 2.1
        y = 1.4
        h1 = h
        w1 = w
    elif angle == 180:
        x = 2
        y = 2
        h1 = w
        w1 = h
    elif angle == 270:
        x = 2.8
        y = 2
        h1 = h
        w1 = w
    else:
        x = 1
        y = 1
        h1 = w
        w1 = h

    if center is None:
        center = (w / x, h / y)

    # Perform the rotation
    M = cv2.getRotationMatrix2D(center, angle, scale)
    rotated1 = cv2.warpAffine(rotated, M, (h1, w1))

    temp_file = tempfile.NamedTemporaryFile(delete=False, suffix='.jpg')
    temp_filename = temp_file.name
    cv2.imwrite(temp_filename, rotated1)
    return temp_filename


### Increase image quality

In [67]:
def process_image_for_ocr(file_path):
    # TODO : Implement using opencv
    temp_filename = set_image_dpi(file_path)
    im_new = remove_noise_and_smooth(temp_filename)
    return im_new

def set_image_dpi(file_path):
    im = Image.open(file_path)
    length_x, width_y = im.size
    factor = max(2, int(IMAGE_SIZE / length_x))
    size = factor * length_x, factor * width_y
    # size = (1800, 1800)
#     im_resized = im.resize(size, Image.ANTIALIAS)
    im_resized = im.resize(size, Image.ANTIALIAS)
    temp_file = tempfile.NamedTemporaryFile(delete=False, suffix='.jpg')
    temp_filename = temp_file.name
    im_resized.save(temp_filename, dpi=(500, 500))
    return temp_filename

def image_smoothening(img):
    ret1, th1 = cv2.threshold(img, BINARY_THREHOLD, 255, cv2.THRESH_BINARY)
    ret2, th2 = cv2.threshold(th1, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    blur = cv2.GaussianBlur(th2, (1, 1), 0)
    ret3, th3 = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    return th3

def remove_noise_and_smooth(file_name):
    img = cv2.imread(file_name, 0)
    filtered = cv2.adaptiveThreshold(img.astype(np.uint8), 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 41,
                                     9)
    kernel = np.ones((1, 1), np.uint8)
    opening = cv2.morphologyEx(filtered, cv2.MORPH_OPEN, kernel)
    closing = cv2.morphologyEx(opening, cv2.MORPH_CLOSE, kernel)
    img = image_smoothening(img)
    or_image = cv2.bitwise_or(img, closing)
    return or_image

In [39]:
rotate_image = rotate(image='../voterid/image_4.jpg') 
temp_filename = set_image_dpi(rotate_image)
im_new = remove_noise_and_smooth(temp_filename)

cv2.imshow('test',im_new)
cv2.waitKey()
cv2.destroyAllWindows()

In [7]:
imdir = '../data/all_images/'
ext = ['png', 'jpeg', 'jpg']    # Add image formats here
files = []
[files.extend(glob.glob(imdir + '*.' + e)) for e in ext]
score = []
print(files)

['../data/all_images\\a_adhar_cropped.jpeg', '../data/all_images\\IMG-20190808-WA0023_cropped.jpeg', '../data/all_images\\IMG-20190808-WA0026_cropped.jpeg', '../data/all_images\\IMG-20190808-WA0032_cropped.jpeg', '../data/all_images\\IMG-20190808-WA0037_cropped.jpeg', '../data/all_images\\IMG-20190808-WA0038_cropped.jpeg', '../data/all_images\\IMG-20190808-WA0039_cropped.jpeg', '../data/all_images\\IMG-20190808-WA0041_cropped.jpeg', '../data/all_images\\image_1.jpg', '../data/all_images\\image_2.jpg', '../data/all_images\\image_3.jpg', '../data/all_images\\image_4.jpg', '../data/all_images\\image_5.jpg', '../data/all_images\\image_8.jpg']


In [68]:
for file in files:
    rotate_image = rotate(image=file) 
    temp_filename = set_image_dpi(rotate_image)
    im_new = remove_noise_and_smooth(temp_filename)
    filename = file.split('\\')[-1].split(".")[0]+"_processed.jpg"
    
    cv2.imwrite('../data/all_images/processed/'+filename,im_new)
    
    
#     cv2.imshow('test',im_new)
#     cv2.waitKey()
#     cv2.destroyAllWindows()